# Demo: Virality on Bluesky

Let's look at how a post is shared and spread on Bluesky

## Normal Bluesky Setup

We'll start by doing our normal steps including these helper functions:

### helper function for atproto links
_NOTE: You don't need to worry about the details of how this works, it just is here to make the code later easier to use._

In [ ]:
import re #load a "regular expression" library for helping to parse text

def get_at_post_link_from_url(url):

    # Extract username and post ID from the URL
    match = re.search(r'https://bsky.app/profile/([^/]+)/post/([^/]+)', url)
    if not match:
        raise ValueError("Invalid Bluesky post URL format.")
    user_handle, post_id = match.groups()

    author_profile = client.app.bsky.actor.get_profile({'actor': user_handle})
    
    # Construct the at:// URI
    post_uri = f"at://{author_profile.did}/app.bsky.feed.post/{post_id}"

    return post_uri


Now we can continue logging in to Bluesky and look through multiple posts.
### load atproto library

In [ ]:
# Load some code called "Client" from the "atproto" library that will help us work with Bluesky
from atproto import Client

### login to Bluesky

In [ ]:
# Login to Bluesky
%run bluesky_keys.py

client = Client(base_url="https://bsky.social")
client.login(handle, password)

## Helper function to display text in an indented box
(You don't need to worry about how this works. This is that function that helps display posts in indented boxes)

In [ ]:
from IPython.display import HTML, Image, display
import html
def display_indented(text, left_margin=0):
    display(
        HTML(
            "<pre style='border:solid 1px;padding:3px;margin-top:3px;margin-bottom:3px;margin-left:"+ str(left_margin) + "px'>" + 
            html.escape(text) + 
            "</pre>"
        )
    )

## Code to look up a post

In [ ]:
post_url = "https://bsky.app/profile/lindsayellis.bsky.social/post/3m4mz5juz3224"
post_at_link = get_at_post_link_from_url(post_url)

# load data for the post we linked (the function lets us load multiple posts)
post_results = client.get_posts([post_at_link])
# get the first result (since we know we only asked about one post)
post = post_results.posts[0]

In [ ]:
## Code to print a post

In [ ]:
display_indented(
    post.record.text + "\n" +
    "-- " + str(post.author.display_name) + " (" + str(post.author.handle) + ")\n" + 
    " (likes: " + str(post.like_count) + 
    ", replies: " + str(post.reply_count) +
    ", reposts: " + str(post.repost_count) + 
    ", quotes: " + str(post.quote_count) + ") - "
)

## Code to print post and quote posts

In [ ]:
display_indented(
    post.record.text + "\n" +
    "-- " + str(post.author.display_name) + " (" + str(post.author.handle) + ")\n" + 
    " (likes: " + str(post.like_count) + 
    ", replies: " + str(post.reply_count) +
    ", reposts: " + str(post.repost_count) + 
    ", quotes: " + str(post.quote_count) + ") - " 
)

quote_posts = client.app.bsky.feed.get_quotes({"uri": post_at_link, "limit": 100}).posts

# sort by quote count, then like count if a tie
sorted_quote_posts = sorted(quote_posts, key=lambda x: (-x.quote_count, -x.like_count))

# display the first top five quotes

for quote_post in sorted_quote_posts[: 5]:
    display_indented(
        quote_post.record.text + "\n" +
        "-- " + str(quote_post.author.display_name) + " (" + str(quote_post.author.handle) + ")\n" + 
        " (likes: " + str(quote_post.like_count) + 
        ", replies: " + str(quote_post.reply_count) +
        ", reposts: " + str(quote_post.repost_count) + 
        ", quotes: " + str(quote_post.quote_count) + ") - ",
        left_margin = 10
    )

## Code to print post and quote posts and quote quote posts

In [ ]:
display_indented(
    post.record.text + "\n" +
    "-- " + str(post.author.display_name) + " (" + str(post.author.handle) + ")\n" + 
    " (likes: " + str(post.like_count) + 
    ", replies: " + str(post.reply_count) +
    ", reposts: " + str(post.repost_count) + 
    ", quotes: " + str(post.quote_count) + ") - " 
)

quote_posts = client.app.bsky.feed.get_quotes({"uri": post_at_link, "limit": 100}).posts

# sort by quote count, then like count if a tie
sorted_quote_posts = sorted(quote_posts, key=lambda x: (-x.quote_count, -x.like_count))

# display the first top five quotes

for quote_post in sorted_quote_posts[: 5]:
    display_indented(
        quote_post.record.text + "\n" +
        "-- " + str(quote_post.author.display_name) + " (" + str(quote_post.author.handle) + ")\n" + 
        " (likes: " + str(quote_post.like_count) + 
        ", replies: " + str(quote_post.reply_count) +
        ", reposts: " + str(quote_post.repost_count) + 
        ", quotes: " + str(quote_post.quote_count) + ") - ",
        left_margin = 10
    )

    quote_quote_posts = client.app.bsky.feed.get_quotes({"uri": quote_post.uri, "limit": 100}).posts

    # sort by quote count, then like count if a tie
    sorted_quote_quote_posts = sorted(quote_quote_posts, key=lambda x: (-x.quote_count, -x.like_count))
    
    for quote_quote_post in sorted_quote_quote_posts[: 5]:
        display_indented(
            quote_quote_post.record.text + "\n" +
            "-- " + str(quote_quote_post.author.display_name) + " (" + str(quote_quote_post.author.handle) + ")\n" + 
            " (likes: " + str(quote_quote_post.like_count) + 
            ", replies: " + str(quote_quote_post.reply_count) +
            ", reposts: " + str(quote_quote_post.repost_count) + 
            ", quotes: " + str(quote_quote_post.quote_count) + ") - ",
            left_margin = 20
        )


In [ ]:
print_post_thread('https://bsky.app/profile/realgdt.bsky.social/post/3lihunicmds2y')